In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy
import matplotlib.pyplot as plt
import kalepy as kale


import matplotlib.cm as cm
from holodeck import plot, utils
from holodeck.constants import YR, MSOL, GYR

# Build Semi-Analytic Model

In [ ]:
dur, cad = 17.5*YR, 0.2*YR
fobs = utils.nyquist_freqs(dur,cad)
fobs_edges = utils.nyquist_freqs_edges(dur,cad)
sam = holo.sam.Semi_Analytic_Model()
# sam = holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e11*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))  # faster version

## Test GWB calculation with default Hard_GW

In [ ]:
# just using gwb() function
gwb = sam.gwb_old(fobs_edges, hard=holo.hardening.Hard_GW, realize=10)
fig = plot.plot_gwb(fobs, gwb)

## Test GWB calculation with Fixed_Time model

In [ ]:
# just using gwb() function

gwb = sam.new_gwb(fobs_edges, hard=holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR), realize=10)
fig = plot.plot_gwb(fobs, gwb)

# Calculate SS and GWB strain with sam.gwb()
This function calls single_sources.ss_gws_redz()
## using Hard_GW
### with params

In [ ]:
hard=holo.hardening.Hard_GW()
assert isinstance(hard, (holo.hardening.Fixed_Time_2PL_SAM, holo.hardening.Hard_GW))

In [ ]:
hc_ss, hc_bg, sspar, bgpar = sam.gwb(fobs_edges, hard=holo.hardening.Hard_GW(), realize = 10, loudest = 5, params = True)

In [ ]:
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

In [ ]:
fig = plot.plot_pars(fobs, sspar, bgpar)

### without params

In [ ]:
hc_ss, hc_bg = sam.gwb(fobs_edges, hard=holo.hardening.Hard_GW(), realize=30, loudest=100, params=False)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)

## using Fixed_Time
### with params

In [ ]:
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
hc_ss, hc_bg, sspar, bgpar = sam.gwb(fobs_edges, hard=hard, realize = 10, loudest = 5, params = True)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)
fig = plot.plot_pars(fobs, sspar, bgpar )

### without params

In [ ]:
hc_ss, hc_bg = sam.gwb(fobs_edges, hard=hard, realize = 10, loudest = 5, params = False)
fig = plot.plot_bg_ss(fobs, hc_bg, hc_ss)